In [0]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0522 18:48:48.383777 139826716571520 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [3]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 8.2MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [5]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'OUTPUT_DIR_NAME'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: OUTPUT_DIR_NAME *****


#Data

In [6]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks


In [0]:
import pandas as pd
train_data = pd.read_csv("train_sarc_proc_bal.csv")
test_data = pd.read_csv("test_sarc_proc_bal.csv")

train.columns

In [0]:
train_responses1 = train_data[['Response 1','Ancestor','Label 1']].rename({'Response 1':'Response','Label 1':'Label'}, axis='columns')
train_responses2 = train_data[['Response 2','Ancestor','Label 2']].rename({'Response 2':'Response','Label 2':'Label'}, axis='columns')
train = pd.concat([train_responses1,train_responses2],sort=False)

test_responses1 = test_data[['Response 1','Ancestor','Label 1']].rename({'Response 1':'Response','Label 1':'Label'}, axis='columns')
test_responses2 = test_data[['Response 2','Ancestor','Label 2']].rename({'Response 2':'Response','Label 2':'Label'}, axis='columns')
test = pd.concat([test_responses1, test_responses2],sort=False)

In [0]:
DATA_COLUMN = 'Response'
LABEL_COLUMN = 'Label'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [-1, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [12]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0522 18:52:03.157849 139826716571520 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0522 18:52:05.228447 139826716571520 saver.py:1483] Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [13]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [14]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 13668


I0522 18:52:07.449356 139826716571520 run_classifier.py:774] Writing example 0 of 13668


INFO:tensorflow:*** Example ***


I0522 18:52:07.454634 139826716571520 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0522 18:52:11.963226 139826716571520 run_classifier.py:466] input_mask: 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0522 18:52:11.965127 139826716571520 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


I0522 18:52:11.966974 139826716571520 run_classifier.py:468] label: 1 (id = 1)


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [20]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2b813f12b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0522 18:52:13.085521 139826716571520 estimator.py:201] Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2b813f12b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [22]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0522 18:52:21.550015 139826716571520 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0522 18:52:24.948029 139826716571520 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 1281 into OUTPUT_DIR_NAME/model.ckpt.


I0522 19:12:02.936779 139826716571520 basic_session_run_hooks.py:594] Saving checkpoints for 1281 into OUTPUT_DIR_NAME/model.ckpt.


INFO:tensorflow:Loss for final step: 0.17483354.


I0522 19:12:14.654412 139826716571520 estimator.py:359] Loss for final step: 0.17483354.


Training took time  0:20:01.533010


Now let's use our test data to see how well our model did:

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [24]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0522 19:12:16.451868 139826716571520 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0522 19:12:20.316567 139826716571520 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 1281: auc = 0.7234292, eval_accuracy = 0.72342926, f1_score = 0.7256843, false_negatives = 457.0, false_positives = 485.0, global_step = 1281, loss = 0.8917152, precision = 0.71981514, recall = 0.73165005, true_negatives = 1218.0, true_positives = 1246.0


I0522 19:13:07.594083 139826716571520 estimator.py:1979] Saving dict for global step 1281: auc = 0.7234292, eval_accuracy = 0.72342926, f1_score = 0.7256843, false_negatives = 457.0, false_positives = 485.0, global_step = 1281, loss = 0.8917152, precision = 0.71981514, recall = 0.73165005, true_negatives = 1218.0, true_positives = 1246.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1281: OUTPUT_DIR_NAME/model.ckpt-1281


I0522 19:13:10.977945 139826716571520 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1281: OUTPUT_DIR_NAME/model.ckpt-1281


{'auc': 0.7234292,
 'eval_accuracy': 0.72342926,
 'f1_score': 0.7256843,
 'false_negatives': 457.0,
 'false_positives': 485.0,
 'global_step': 1281,
 'loss': 0.8917152,
 'precision': 0.71981514,
 'recall': 0.73165005,
 'true_negatives': 1218.0,
 'true_positives': 1246.0}

Now let's write code to make predictions on new sentences:

In [0]:
def getPrediction(in_sentences):
  labels = ["Non-sarcastic", "Sarcastic"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = -1) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = [
  "Good luck with that!",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "I don't think that's a good idea"
]

In [27]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4


I0522 19:13:11.019611 139826716571520 run_classifier.py:774] Writing example 0 of 4


INFO:tensorflow:*** Example ***


I0522 19:13:11.023629 139826716571520 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0522 19:13:15.504606 139826716571520 saver.py:1270] Restoring parameters from OUTPUT_DIR_NAME/model.ckpt-1281


INFO:tensorflow:Running local_init_op.


I0522 19:13:17.160218 139826716571520 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0522 19:13:17.305893 139826716571520 session_manager.py:493] Done running local_init_op.


Voila! We have a sentiment classifier!

In [28]:
predictions

[('Good luck with that!',
  array([-1.9468852 , -0.15398827], dtype=float32),
  'Sarcastic'),
 ('The acting was a bit lacking',
  array([-2.113361  , -0.12877831], dtype=float32),
  'Sarcastic'),
 ('The film was creative and surprising',
  array([-4.3373766 , -0.01315698], dtype=float32),
  'Sarcastic'),
 ("I don't think that's a good idea",
  array([-0.03443514, -3.3858461 ], dtype=float32),
  'Non-sarcastic')]

In [92]:
test_responses2.head()

Response  ...    probs2
0  oooh baby you caught me red handed creepin' on...  ... -1.825755
1                                   welcome to china  ... -0.137434
2                 what's the vegas line on this one?  ... -3.761858
3  to quote william gibson: "she stared directly ...  ... -4.745414
4  i wonder why it isn't thought men should have ...  ... -0.009787

[5 rows x 7 columns]

In [97]:
test_responses1[test_responses1.probs2>0]

Empty DataFrame
Columns: [Response, Ancestor, Label, probs, labels, probs1, probs2]
Index: []

In [88]:
def getPredictions(df, data="Response", label="Label"):
  print(label_list)
  labels = [-1, 1]
  input_examples = df.apply(lambda x:bert.run_classifier.InputExample(guid=None, 
                                                                 text_a = x[data], 
                                                                 text_b = None, label = x[label]),axis=1)
  
  input_features = bert.run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = bert.run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = list(estimator.predict(predict_input_fn))
  probs = np.array([pred['probabilities'] for pred in predictions])
  preds = [int(labels[pred['labels']]) for pred in predictions]
  df['probs1'] = pd.Series(probs[:,0])
  df['probs2'] = pd.Series(probs[:,1])
  df['labels'] = pd.Series(preds)
  
  return df
#   return [(prediction['labels']) for prediction in zip(predictions)]

getPredictions(test_responses1)
getPredictions(test_responses2)


[-1, 1]
INFO:tensorflow:Writing example 0 of 1703


I0522 19:55:17.256483 139826716571520 run_classifier.py:774] Writing example 0 of 1703


INFO:tensorflow:*** Example ***


I0522 19:55:17.260426 139826716571520 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Running local_init_op.


I0522 19:55:46.117746 139826716571520 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0522 19:55:46.279045 139826716571520 session_manager.py:493] Done running local_init_op.


Response  ...    probs2
0     oooh baby you caught me red handed creepin' on...  ... -1.825755
1                                      welcome to china  ... -0.137434
2                    what's the vegas line on this one?  ... -3.761858
3     to quote william gibson: "she stared directly ...  ... -4.745414
4     i wonder why it isn't thought men should have ...  ... -0.009787
5                it's ok guys it'll make romney richer.  ... -0.015269
6     that is a blatant violation of the law on so m...  ... -4.545376
7                           you sure that wasn't "boy"?  ... -0.678455
8                                 bootstraps of course.  ... -0.554700
9              sounds like tuesday night at the apollo.  ... -4.375214
10                         yeah, but how do they taste?  ... -0.018111
11    i bet they'll use roads to get there, too!...p...  ... -3.350153
12    this is clearly fake, republicans don't use te...  ... -0.006509
13    this may have already between said, but who th...  ... -5.867435
14         it's ok because they only kept the metadata!  ... -0.012172
15    it's a pipeline rupture most likely he will lo...  ... -0.508334
16    i think it would be hilarious if the gop unvei...  ... -5.087144
17    it's not a war on drugs, it's a war on persona...  ... -5.304625
18    yeah because govt is soooo responsible with ou...  ... -0.004103
19    i bet obama is going to call for socialized po...  ... -0.118265
20                               baby steps, baby steps  ... -2.653021
21    it boils down to the following question: can a...  ... -4.954022
22    maybe this would lead to dc statehood becoming...  ... -4.192878
23    my state has tons of oil, therefore climate ch...  ... -0.011839
24    the campaign promise will be broken when she d...  ... -1.433396
25    i am very happy that those republicans are red...  ... -0.013418
26    so if a professor at a college receiving feder...  ... -4.064860
27                       can't say they're not exciting  ... -1.081997
28                       because this one wears glasses  ... -0.375123
29    we're also not going to be a country that tole...  ... -5.353403
...                                                 ...  ...       ...
1673                          lincoln was a republican.  ... -3.894554
1674                                   not us politics.  ... -4.419440
1675                 trump keeps up his winning streak.  ... -4.565708
1676  poor, white people in the usa are particular g...  ... -3.467941
1677  this guy sprints to the white house yesterday ...  ... -2.713221
1678       today i learned that california is 31% nuts.  ... -0.510522
1679               the news is real, the story is fake.  ... -0.084778
1680                     12d chess, the mans got a plan  ... -0.293880
1681  there were invisible quote marks around those ...  ... -0.018850
1682  of course, because campaign managers have no r...  ... -0.007147
1683                         trump: challenge accepted.  ... -3.796689
1684                      it's an _entertainment_ show.  ... -3.659964
1685  sometimes you wanna go- where no one can call ...  ... -0.680699
1686                     dopey paul ryan bends the knee  ... -5.540935
1687                                               duh.  ... -4.052082
1688                                  texas, take note.  ... -4.092294
1689  nah dude the free market will sort it out, cor...  ... -0.008722
1690                                            no way!  ... -0.053178
1691   who knew tall concrete walls would be expensive?  ... -0.010816
1692  yeah, who would have figured that a rich billi...  ... -0.003748
1693                                  "he was on to us"  ... -0.851166
1694  every photo i see of this guy looks like regre...  ... -5.277926
1695  probably not the best time for her to be speak...  ... -4.455033
1696  for all we know, he might never claim illegal ...  ... -0.375842
1697  man i'm sure glad the republican's learned the...  ... -0.003981


In [121]:

# train_pred_labels = 2*(classifier.predict(train_responses1)>classifier.predict(train_responses2))-1
# train_expect_labels = train_data['Label 1']

test_pred_labels = 2*(np.exp(test_responses1['probs2'])>np.exp(test_responses2['probs2']))-1
test_expect_labels = test_data['Label 1']


# print('\tTrain acc: ', (train_pred_labels == train_expect_labels).sum() / train_pred_labels.shape[0])
print('\tTest acc: ', (test_pred_labels == test_expect_labels).sum() / test_pred_labels.shape[0])

	Test acc:  0.7962419260129184


In [108]:
test_responses1.head()

Response  ...    probs2
0  and we're upset since the democrats would *nev...  ... -0.006981
1  til, voluntary slavery by starving people who'...  ... -2.379236
2  that'd be great if only ron paul wasn't a chri...  ... -0.016866
3            but, but...lowering taxes creates jobs!  ... -0.014451
4                                            whores!  ... -0.490472

[5 rows x 7 columns]

In [117]:
np.exp(test_responses1['probs1'])*0

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
1673    0.0
1674    0.0
1675    0.0
1676    0.0
1677    0.0
1678    0.0
1679    0.0
1680    0.0
1681    0.0
1682    0.0
1683    0.0
1684    0.0
1685    0.0
1686    0.0
1687    0.0
1688    0.0
1689    0.0
1690    0.0
1691    0.0
1692    0.0
1693    0.0
1694    0.0
1695    0.0
1696    0.0
1697    0.0
1698    0.0
1699    0.0
1700    0.0
1701    0.0
1702    0.0
Name: probs1, Length: 1703, dtype: float32

In [120]:
list(zip(np.exp(test_responses1['probs1'])*0+1*np.exp(test_responses1['probs2']),np.exp(test_responses2['probs1'])*0+1*np.exp(test_responses2['probs2']),np.exp(test_responses1['probs1'])*0+1*np.exp(test_responses1['probs2'])>np.exp(test_responses2['probs1'])*0+1*np.exp(test_responses2['probs2']),train_data['Label 1']))

[(0.9930434226989746, 0.1610960215330124, True, 1),
 (0.0926213338971138, 0.8715921640396118, False, 1),
 (0.9832752346992493, 0.023240525275468826, True, 1),
 (0.9856531620025635, 0.008691460825502872, True, 1),
 (0.6123374700546265, 0.9902607798576355, False, -1),
 (0.9889251589775085, 0.9848472476005554, True, -1),
 (0.0055563789792358875, 0.010616176761686802, False, 1),
 (0.9875761270523071, 0.5074001550674438, True, 1),
 (0.19164417684078217, 0.5742444396018982, False, 1),
 (0.9915619492530823, 0.012585447169840336, True, -1),
 (0.07511938363313675, 0.9820523858070374, False, -1),
 (0.00808774959295988, 0.0350789949297905, False, 1),
 (0.9886903762817383, 0.9935120940208435, False, 1),
 (0.988135814666748, 0.002830124692991376, True, -1),
 (0.005193168297410011, 0.9879015684127808, False, -1),
 (0.03830774873495102, 0.6014970541000366, False, -1),
 (0.9841346144676208, 0.006175633054226637, True, -1),
 (0.9651614427566528, 0.004968560766428709, True, 1),
 (0.06829016655683517, 0.

In [86]:
l = [[1,2],[3,4]]
import numpy as np
np.array(l)[:,1]

array([2, 4])

In [87]:
test_responses1['probs'][:,1]

ValueError: ignored